In [ ]:
! export PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python

In [ ]:
import os
import sys
api_model_path = "" # directory of api model path
os.environ['PYTHONPATH']+=f":{api_model_path}"
sys.path.append(f"{api_model_path}/research")

In [ ]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

### Generate tf-record

In [ ]:

SCRIPTS_PATH = './scripts'
PRETRAINED_MODEL_PATH = "./pre-trained-models"
APIMODEL_PATH = api_model_path
ANNOTATION_PATH = "./my-model/label_map.pbtxt"
TRAINING_DATA_PATH = "./training-data"
TESTING_DATA_PATH = "./test-data"
MODEL_PATH = "./my-model"
CONFIG_PATH ='./my-model/pipeline.config'
CHECKPOINT_PATH = "./my-model"

In [ ]:
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {TRAINING_DATA_PATH} -l {ANNOTATION_PATH} -o {MODEL_PATH + '/train.record'}
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x{TESTING_DATA_PATH} -l {ANNOTATION_PATH} -o {MODEL_PATH + '/test.record'}

### Define Model

In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [ ]:
pipeline_config.model.ssd.num_classes = 10
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [MODEL_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [MODEL_PATH + '/test.record']

In [ ]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

### Train Model

In [ ]:
print("""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=25000""".format(APIMODEL_PATH, MODEL_PATH,"my-model",MODEL_PATH,"my-model"))

In [ ]:
! python {api_model_path}/research/object_detection/model_main_tf2.py --model_dir=./my-model --pipeline_config_path=./my-model/pipeline.config --num_train_steps=25000

### Test Model

In [ ]:

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [ ]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-26')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
def load_image_into_numpy_array(path):
    return np.array(Image.open(path))

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap("my-model"+'/label_map.pbtxt')

In [ ]:

image_np = load_image_into_numpy_array('image.jpg')
input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
detections = detect_fn(input_tensor)
    

label_id_offset = 1


my_classes = detections['detection_classes'][0].numpy() + label_id_offset
my_scores = detections['detection_scores'][0].numpy()

min_score = 0.5
output = {}
for index,value in enumerate(my_classes):
    if my_scores[index] > min_score:
        output['class'] =  category_index.get(int(value))
        output['score'] = float(my_scores[index])
print(output)